In [377]:
# Relevant imports
import torch
import sys
import matplotlib.pyplot as plt
import numpy as np
from kan import KAN

# Set the seed for reproducibility
torch.manual_seed(42)

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../../utils')
from upinn import UPINN
from architectures import FNN, StackedNN, ResNet, count_parameters
from utils import RAD_sampler, sample_collocation_points
from NavierStokesData import NavierStokesData

In [ ]:
# Data points
data = NavierStokesData(samplesize=2000, noise_level=0.01)
Zd = data.Zd
Ud = data.Ud

In [380]:
# Collocation points
N_coll = 10000
Xc = sample_collocation_points(N_coll, 3, [1, -2, 0], [8, 2, 20], method='sobol')

In [381]:
def compute_grad(outputs, inputs):
    return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True, retain_graph=True)[0]

class NavierStokes(torch.nn.Module):
    def __init__(self, lambda1, lambda2):
        super(NavierStokes, self).__init__()
        self.lambda1 = lambda1
        self.lambda2 = lambda2


    def forward(self, Z, U):

        psi = U[:, 0:1]
        p = U[:, 1:2]

        U_z = compute_grad(psi, Z)

        psi_x = U_z[:, 0:1]
        psi_y = U_z[:, 1:2]

        u = psi_y
        v = -psi_x

        u_z = compute_grad(u, Z)
        u_x = u_z[:, 0:1]
        u_y = u_z[:, 1:2]
        u_t = u_z[:, 2:3]

        v_z = compute_grad(v, Z)
        v_x = v_z[:, 0:1]
        v_y = v_z[:, 1:2]
        v_t = v_z[:, 2:3]

        p_z = compute_grad(p, Z)
        p_x = p_z[:, 0:1]
        p_y = p_z[:, 1:2]

        u_xx = compute_grad(u_x, Z)[:, 0:1]
        u_yy = compute_grad(u_y, Z)[:, 1:2]

        v_xx = compute_grad(v_x, Z)[:, 0:1]
        v_yy = compute_grad(v_y, Z)[:, 1:2]

        f = u_t + self.lambda1 * (u * u_x + v * u_y) + p_x # - self.lambda2 * (u_xx + u_yy)
        g = v_t + self.lambda1 * (u * v_x + v * v_y) + p_y # - self.lambda2 * (v_xx + v_yy)

        return torch.cat([f, g], dim=1)

lambda1 = 1.0
lambda2 = 0.01

N = NavierStokes(lambda1, lambda2)

In [ ]:
class NavierStokesUPINN(UPINN):
    
    def score(self):

        Xtest = data.Zd_full.requires_grad_(True)

        U_pred = self.u(Xtest)
        psi_z = compute_grad(U_pred[:, 0], Xtest)
        psi_y = psi_z[:, 1:2]
        psi_x = psi_z[:, 0:1]

        data_pred = torch.cat([psi_y, -psi_x], dim=1)
        data_true = data.Ud_full[:, 0:2]

        L2_rel_error = torch.sqrt(torch.mean((data_pred - data_true)**2) / torch.mean(data_true**2))

        print(torch.nn.MSELoss()(data_pred, data_true).item())

        return L2_rel_error.item()
    

    def score_residual(self):
        
        # Expected residual
        x = torch.linspace(1, 8, 30)
        y = torch.linspace(-2, 2, 30)
        t = torch.linspace(0, 20, 30)

        Z = torch.cartesian_prod(x, y, t).requires_grad_(True)

        U_pred = self.u(Z)
        psi_z = compute_grad(U_pred[:, 0], Z)
        psi_y = psi_z[:, 1:2]
        psi_x = psi_z[:, 0:1]

        u_pred = psi_y
        v_pred = -psi_x

        u_z = compute_grad(u_pred, Z)
        u_x = u_z[:, 0:1]
        u_y = u_z[:, 1:2]

        v_z = compute_grad(v_pred, Z)
        v_x = v_z[:, 0:1]
        v_y = v_z[:, 1:2]

        u_xx = compute_grad(u_x, Z)[:, 0:1]
        u_yy = compute_grad(u_y, Z)[:, 1:2]

        v_xx = compute_grad(v_x, Z)[:, 0:1]
        v_yy = compute_grad(v_y, Z)[:, 1:2]

        res_exp_f = - lambda2 * (u_xx + u_yy)
        res_exp_g = - lambda2 * (v_xx + v_yy)
        res_exp = torch.cat([res_exp_f.reshape(-1,1), res_exp_g.reshape(-1,1)], dim=1)

        # Predicted residual
        res_pred = self.F(self.F_input(Z, U_pred))

        # Residual MSE
        # residual_loss = torch.mean((res_exp - res_pred)**2)
        print(torch.mean((res_exp - res_pred)**2).item())
        print(torch.corrcoef(torch.cat([res_exp[:, 0], res_pred[:, 0]])).item())
        print(torch.corrcoef(torch.cat([res_exp[:, 1], res_pred[:, 1]])).item())

        # temp = data.get_quantities(self)

        # f_res_true = temp["f_res_true"]
        # g_res_true = temp["g_res_true"]

        # f_res_exp = temp["f_res_exp"]
        # g_res_exp = temp["g_res_exp"]

        # f_res_pred = temp["f_res_pred"]
        # g_res_pred = temp["g_res_pred"]

        # print('True:', np.sqrt((np.mean((f_res_true - f_res_pred)**2) + np.mean((g_res_true - g_res_pred)**2)) / (np.mean(f_res_true**2) + np.mean(g_res_true**2)))) 

        L2_rel_error = torch.sqrt(torch.mean((res_exp - res_pred)**2) / torch.mean(res_exp**2))
        return L2_rel_error.item()


    def data_loss(self):

        if self.data_points is not None:
            
            self.data_points[0].requires_grad_(True)

            Ud = self.u(self.data_points[0])

            psi_z = compute_grad(Ud[:, 0], self.data_points[0])
            psi_y = psi_z[:, 1:2]
            psi_x = psi_z[:, 0:1]

            # data_pred = torch.cat([psi_y, -psi_x], dim=1)
            # data_loss = torch.mean((data_pred - self.data_points[1][:, 0:2])**2)

            data_pred = torch.cat([psi_y, -psi_x, Ud[:, 1:2]], dim=1)
            data_loss = torch.mean((data_pred - self.data_points[1][:, 0:3])**2)

            # self.log.setdefault("lambda1", []).append(lambda1.item())
            # self.log.setdefault("lambda2", []).append(lambda2.item())

        else: data_loss = torch.tensor(0.0)

        return data_loss
    
    def F_input(self, Z, U):
        
        psi = U[:, 0:1]
        
        U_z = compute_grad(psi, Z)

        psi_x = U_z[:, 0:1]
        psi_y = U_z[:, 1:2]

        u = psi_y
        v = -psi_x

        if not self.inductive_bias:
            return torch.cat([Z, u, v], dim=1)
        else:

            u_z = compute_grad(u, Z)
            u_x = u_z[:, 0:1]
            u_y = u_z[:, 1:2]

            v_z = compute_grad(v, Z)
            v_x = v_z[:, 0:1]
            v_y = v_z[:, 1:2]

            u_xx = compute_grad(u_x, Z)[:, 0:1]
            u_yy = compute_grad(u_y, Z)[:, 1:2]

            v_xx = compute_grad(v_x, Z)[:, 0:1]
            v_yy = compute_grad(v_y, Z)[:, 1:2]

            u_xy = compute_grad(u_x, Z)[:, 1:2]
            v_xy = compute_grad(v_x, Z)[:, 1:2]

            return torch.cat([u, v, u_x, u_y, v_x, v_y, u_xy, v_xy, u_xx, u_yy, v_xx, v_yy], dim=1)



    # def refine_collocation_points(self):
    #     N = 10*N_coll
    #     D = N_coll
    #     k = 0.5
    #     c = 0.1

    #     device = self.device
    #     Xc = sample_collocation_points(N, 3, lb=[1, -2, 0], ub=[8, 2, 20], method='sobol').requires_grad_(True).to(device)

    #     self.to(device)

    #     # Compute the residual
    #     U = self.u(Xc)
    #     residual = torch.sum(torch.abs(self.F(self.F_input(Xc, U)) + self.N(Xc, U)), dim=1)

    #     self.to('cpu')

    #     self.collocation_points = RAD_sampler(Xc, residual, D, k, c) # RAD
    


In [383]:
hidden_u = [64] * 8

u = FNN(
    dims=[3, *hidden_u, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Identity(),
)

hidden_F = [64] * 8

F = FNN(
    dims=[5, *hidden_F, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Identity(),
)

In [372]:
# u = ResNet(
#     input_dim=3, hidden_dim=40, output_dim=2, num_blocks=4, block_size=2,
# )

# F = ResNet(
#     input_dim=5, hidden_dim=40, output_dim=2, num_blocks=4, block_size=2,
# )

In [373]:
# u1 = FNN(
#     dims=[3, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# u2 = FNN(
#     dims=[3, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# u = StackedNN([u1, u2])

# F1 = FNN(
#     dims=[5, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# F2 = FNN(
#     dims=[5, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# F = StackedNN([F1, F2])

In [374]:
# hidden = [4] * 4

# u = KAN([3, *hidden, 2])
# F = KAN([5, *hidden, 2])

In [375]:
model = NavierStokesUPINN(u, N, F, data_points=(Zd, Ud), collocation_points=Xc)
model.inductive_bias = False

[Info]: Initializing UPINN model


In [29]:
model.softadapt_kwargs = dict(beta=0.05, loss_weigthed=False)

In [31]:
adam = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(adam, step_size=500, gamma=0.8)
lbfgs = torch.optim.LBFGS(model.parameters(), lr=1e-1)

model.scheduler = scheduler

In [ ]:
model.train_loop(2000, optimizer=adam)